# Calling Web Application Programming Interfaces (APIs)

Many of the specifications we need (increasingly so) are not totally static.  Relying on a static file for importing specifications leaves us open to: 
* missing out on updates
* inability to link/reuse 


Many of the core resources for Clinical Research standards management are available via the web using Universal Resource Locations (URL).  

In general there is a request-response pattern for accessing web resources; we make a request to the service and get a response back.  Checking the response to make sure the request worked is key

In [1]:
# import a library for making web requests
import requests

# make a simple request

response = requests.get("https://cdisc.org")

# The response object is how you work out whether your request worked. 

## A Web Services Response 

The response object tells us how the request proceeeded; key attributes:
* `status_code` - the status code returned by the web server
* `text` - the text of the response
* `json` - if the request returns a JSON structure, the JSON can be automatically converted 

In [2]:
# let's look at the response
print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    print("All ok")


Status Code: 200
All ok


In [3]:
print("Let's look at what the website states it sent to the client")

print(f"Content-type: {response.headers.get('Content-type')}")

print("Content, truncated: ", response.text[:300])

Let's look at what the website states it sent to the client
Content-type: text/html; charset=UTF-8
Content, truncated:  <!DOCTYPE html>
<html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct:


In [4]:
# What happens when we request a website or page that doesn't exist?

bad_guy = requests.get("https://cdisc.org/say-no-to-xpt")

print(f"Requesting {bad_guy.url} got a status code of {bad_guy.status_code}")

print("404 is the HTTP Status code for Not Found")

Requesting https://www.cdisc.org/say-no-to-xpt got a status code of 404
404 is the HTTP Status code for Not Found


In [5]:
# Always check your return codes 

if not 200 <= bad_guy.status_code <= 300:
    print("Request failed")


Request failed


In [6]:
# Let's try and pull a more prevalent data transfer format, namely FHIR

valuesets = requests.get("https://fhir.nhs.uk/ValueSet")


In [7]:
# lets inspect the valuesets response
print(f"Requesting {valuesets.url} got a status of {valuesets.status_code}")
print(f"Content type is {valuesets.headers.get('Content-type')}")

Requesting https://fhir.nhs.uk/ValueSet got a status of 200
Content type is application/xml+fhir;charset=utf-8


In [8]:
print("Response content (truncated)", valuesets.text[:300])

Response content (truncated) <Bundle xmlns="http://hl7.org/fhir"><id value="c6aa3c91-6280-4282-ac1a-a009d1afea2f"></id><type value="searchset"></type><total value="31"></total><link><relation value="self"></relation><url value="http://fhir.nhs.uk/ValueSet"></url></link><link><relation value="next"></relation><url value="http://


In [9]:
# XML is a popular exchange format, but it's not too fun to process -> let's request a JSON response

# the request has a set of Headers that communicate directly to the Server, the set of headers is stand
valuesets_json = requests.get("https://fhir.nhs.uk/ValueSet", headers={"Accept": "application/fhir+json"})

In [10]:
# let's inspect the valuesets response
print(f"Requesting {valuesets_json.url} got a status of {valuesets_json.status_code}")
print(f"Content type is {valuesets_json.headers.get('Content-type')}")

Requesting https://fhir.nhs.uk/ValueSet got a status of 200
Content type is application/json+fhir;charset=utf-8


In [11]:
# We can access the data as a python dict

valueset_dict = valuesets_json.json()
print(f"Keys: {valueset_dict.keys()}")

Keys: dict_keys(['resourceType', 'id', 'type', 'total', 'link', 'entry'])


In [12]:
print(f"ValueSet has {len(valueset_dict.get('entry'))} values")

ValueSet has 5 values


In [13]:
for vs in valueset_dict.get('entry'):
    resource = vs.get('resource')
    print(f"{resource.get('id')} -> {resource.get('status')}")


document-type-codes-snct-1 -> active
spine-error-or-warning-code-1 -> active
CareConnect-LanguageAbilityMode-1 -> active
message-event-1-0 -> active
spine-chargeable-status-1 -> draft


Now we know how to query web APIs, how to anticipate and handle errors and how to access the data.  Let's move on to a relevant web API to look at querying this using Python